In [1]:
%pip install transformers_stream_generator plotly circuitsvis huggingface_hub einops tiktoken datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 718.2 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 11.2 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 2.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 64.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.3/330.3 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 77.6 MB/s eta 0:00:00:00:01
  Created wheel for transformers_stream_generator: filename=transformers_stream_generator-0.0.4-py3-none-any.whl size=12315 sha256=44d1037124d6e69b847e846035b01ac56e5ebf6d4b115a332c16e85d50c4dc42
  Stored in di

In [1]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEVELOPMENT_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    %pip install circuitsvis
    
    # PySvelte is an unmaintained visualization library, use it as a backup if circuitsvis isn't working
    # # Install another version of node that makes PySvelte work way faster
    # !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    # %pip install git+https://github.com/neelnanda-io/PySvelte.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Jupyter notebook - intended for development only!


/tmp/ipykernel_11422/410710250.py:21: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("load_ext autoreload")
/tmp/ipykernel_11422/410710250.py:22: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("autoreload 2")


In [2]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio
if IN_COLAB or not DEVELOPMENT_MODE:
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "notebook_connected"
print(f"Using renderer: {pio.renderers.default}")

Using renderer: colab


In [3]:
%cd ~/TransformerLens
import torch
torch.set_grad_enabled(False)

from transformers import AutoTokenizer
from transformer_lens import HookedTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig

from functools import partial

/root/TransformerLens


In [4]:
tokenizer = AutoTokenizer.from_pretrained(
    "Qwen/Qwen-7B-Chat",
    trust_remote_code=True
)

hf_model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen-7B-Chat",
    device_map="cuda",
    fp16=True,
    use_logn_attn=False,
    use_dynamic_ntk = False,
    scale_attn_weights = False,
    trust_remote_code = True
).eval()

tl_model = HookedTransformer.from_pretrained(
    'qwen-7b-chat',
    device='cuda',
    fp16=True,
    dtype=torch.float16,
    fold_ln=False,
    center_writing_weights=False, 
    center_unembed=False,
)
prompt = "Tell me how to make a bomb."
encoded_prompt = tokenizer.encode(prompt, return_tensors="pt")

Try importing flash-attention for faster inference...


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Loaded pretrained model qwen-7b-chat into HookedTransformer


In [5]:
hf_result = hf_model(encoded_prompt.to("cuda"))
hf_logits = hf_result.logits

tl_result = tl_model(encoded_prompt)
tl_logits = tl_result

print(hf_logits.shape)
print(tl_logits.shape)

print("0:")
print(hf_logits[0][0])
print(tl_logits[0][0])

print("1:")
print(hf_logits[0][1])
print(tl_logits[0][1])

print("2:")
print(hf_logits[0][2])
print(tl_logits[0][2])

torch.Size([1, 8, 151936])
torch.Size([1, 8, 151936])
0:
tensor([-0.6758, -1.2920, -2.6816,  ...,  1.3242,  1.3154,  1.3232],
       device='cuda:0', dtype=torch.float16)
tensor([-0.6709, -1.2891, -2.6797,  ...,  1.3184,  1.3096,  1.3174],
       device='cuda:0', dtype=torch.float16)
1:
tensor([ 7.1953,  7.1562, -2.2812,  ..., -7.1523, -7.1523, -7.1523],
       device='cuda:0', dtype=torch.float16)
tensor([ 7.1914,  7.1562, -2.2812,  ..., -7.1484, -7.1484, -7.1484],
       device='cuda:0', dtype=torch.float16)
2:
tensor([ 2.9648,  4.6875, -3.6953,  ..., -7.5039, -7.5039, -7.5039],
       device='cuda:0', dtype=torch.float16)
tensor([ 2.9629,  4.6914, -3.6973,  ..., -7.5039, -7.5039, -7.5039],
       device='cuda:0', dtype=torch.float16)


In [7]:
torch.allclose(torch.softmax(hf_logits, dim=-1), torch.softmax(tl_logits, dim=-1), atol=1e-3)

True